# Advanced Challenge: Randomized Benchmarking

In this challenge, you will implement randomised benchmarking, a protocol to determine the 
average gate fidelity of a quantum process.



In [1]:
from pyquil import Program
from pyquil.api import get_qc, WavefunctionSimulator, local_qvm
from pyquil.gates import *
import numpy as np
import os, inspect, sys

import sys
sys.path.insert(0, 'tests/')

# from test_single_qubit_unitaries import *

# from auxiliary_functions.auxiliary_unitaries import plus_prep, minus_prep


In [2]:
qc_name = '2q-qvm'
with local_qvm():
    qc = get_qc(qc_name)

qubits = qc.qubits()

In [3]:
#Firstly define the pulse lengths:
    
lengths = [5, 10, 20, 30, 50, 100] 

In [4]:
import random

def random_clifford(circuit, clifford_gates,random_seed):
    
    random.seed(random_seed)
    random_clifford_gate = random.choice(clifford_gates)

    circuit += random_clifford_gate(qubits[0])
    
    return circuit, random_clifford_gate


Next, define a function to apply a sequence of these random unitaries to the initial state

In [5]:
def random_clifford_sequence(circuit, random_seed, clifford_gates, length):
    
    random_seed = random_seed
    random_gates = []
    for length in range(length):
        circuit, random_clifford_gate = random_clifford(circuit, clifford_gates, random_seed*length)
        random_gates.append(random_clifford_gate)     # keep track of gate, run same sequence

    for gate in random_gates[::-1]: # Invert sequence and apply gates in reverse order (clifford are self-inverse)
        circuit += gate(qubits[0])
    return circuit

# circuit = Program()
# clifford_gates = [H, Z, X]

# random_seed = 1
# length = 4
# circuit = random_clifford_sequence(circuit, random_seed, clifford_gates, length)

Now take that circuit of random cliffords, and measure the fidelity of the final state remaining in the initial state. 

The final state is denoted $|\psi\rangle$, and the initial state was the computational basis state, $|0\rangle$.

Therefore the fidelity is:

\begin{align}
F = |\langle 0 |\psi \rangle|^2 = \Pr(final state = |0\rangle)
\end{align}

So we can estimate this by measuring the qubit a number of times, and counting the number of times we get the '0' ($|0\rangle$) outcome.

# Task:
## Apply a measurement on the single qubit and compute the fidelity.

In [6]:
def compute_fidelity(circuit, random_seed, clifford_gates, length, num_shots):
    
    circuit = random_clifford_sequence(circuit, random_seed, clifford_gates, length)
    
    ro = circuit.declare('ro', 'BIT', 1)
    circuit += MEASURE(qubits[0], ro[0])
    circuit.wrap_in_numshots_loop(num_shots)
    
    executable = qc.compile(circuit)
    results = qc.run(executable)
    
    prob_zero = list(results).count([0])/num_shots 
     
    
    return prob_zero


circuit = Program()
clifford_gates = [H, Z, X]

random_seed = 1
length = 4
num_shots = 1000

fidelity = compute_fidelity(circuit, random_seed, clifford_gates, length, num_shots)

print(fidelity)

1.0


In [11]:
def compute_avg_fidelity(random_seeds, clifford_gates, lengths, num_shots):
    average_fidelities = np.zeros(len(lengths)) # Array of average fidelities for each sequence length
    
    num_seq_repeats = len(random_seeds)
    for length in range(len(lengths)):
        fidelity = np.zeros(num_seq_repeats)
        seq_len = lengths[length]
        for repeat in range(num_seq_repeats):
            circuit = Program()
            fidelity[repeat] = compute_fidelity(circuit, random_seed, clifford_gates, seq_len, num_shots)
#             print(fidelity)
        average_fidelities[length] = (1/num_seq_repeats)*np.sum(fidelity)
    
    return average_fidelities

sequence_length_repeats = 10 # Number of repeats of a particular sequence length

random_seeds = np.arange(0, 1000, sequence_length_repeats)
num_shots = 10
clifford_gates = [H, Z, X]

average_fidelities = compute_avg_fidelity( random_seeds, clifford_gates, lengths, num_shots)
print('The sequence lenghts are:', lengths)
print('\nThe average fidelities are:' , average_fidelities)
# plot avg_fid_per_lenth vs lenght
# check if you get 1


# Noisy_gates = []

The sequence lenghts are: [5, 10, 20, 30, 50, 100]

The average fidelities are: [1. 1. 1. 1. 1. 1.]
